In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import os

# Отключаем W&B
os.environ["WANDB_DISABLED"] = "true"

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", num_labels=2)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        
        tokenized_inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            is_split_into_words=True
        )
        
        # Форматирование меток
        label_ids = [-100] * len(tokenized_inputs["input_ids"][0])
        for i, label in enumerate(labels[:self.max_length]):
            label_ids[i] = label

        tokenized_inputs["labels"] = torch.tensor(label_ids)
        return {key: val.squeeze() for key, val in tokenized_inputs.items()}

# Пример данных
texts = [
    ["Это", "первый", "текст", ".", "Следующий", "текст", "начинается", "с", "этого", "слова", "."],
    ["Начало", "нового", "текста", ".", "И", "ещё", "один", "пример", "."]
]
labels = [
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 0, 0, 0]
]

# Разделение данных на тренировочные и валидационные
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_strategy="steps",  # Логгирование выполняется через каждый logging_steps шаг
    evaluation_strategy="epoch",  # Оценка в конце каждой эпохи
    save_steps=100,
    logging_steps=10,
    logging_first_step=True, 
)


# Функция вычисления метрик
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    preds = np.argmax(pred.predictions, axis=2).flatten()

    # Оставляем только валидные токены (без -100)
    valid_indices = labels != -100
    labels = labels[valid_indices]
    preds = preds[valid_indices]

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Использование Trainer для обучения с валидацией
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Запуск обучения
trainer.train()


Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.727400,2.648852,0.750000,0.000000,0.000000,0.000000
2,0.727400,1.430057,0.750000,0.000000,0.000000,0.000000
3,0.727400,0.982530,0.750000,0.000000,0.000000,0.000000


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=3, training_loss=0.7238843639691671, metrics={'train_runtime': 17.2573, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.174, 'total_flos': 783890270208.0, 'train_loss': 0.7238843639691671, 'epoch': 3.0})